In [1]:
import numpy as np

In [2]:
from Actfunc import*
from init import*
from Loss import*

MSE:
6.5
Softmax:
[5.60279546e-09 1.52299771e-08 9.99999825e-01 4.13993700e-08
 1.12535155e-07]
Cross:
1.747673127945547e-07


In [3]:
Relu(np.array([-2,0,3]))

array([0., 0., 3.])

In [4]:
class Layer:


    def __init__(self,dim_in,dim_out,init_fn,act_fn=Relu):
        self.weights=init_fn(dim_in,dim_out)
        self.bias=np.reshape(init_fn(1,dim_out),-1)
        self.act_fn=act_fn


    def fw(self,input):
        output=self.weights@input + self.bias
        if self.act_fn:
            output=self.act_fn(output)
        return output    




In [5]:
class FFNN:

    def __init__(self,input_size,hidden_sizes,output_size, init_fn,act_fn):
        self.layers=[]
        self.layers.append(Layer(dim_in=input_size,dim_out=hidden_sizes[0],init_fn=init_fn,act_fn=act_fn))
        for i in range(1,len(hidden_sizes)):
            self.layers.append(Layer(dim_in=hidden_sizes[i-1],dim_out=hidden_sizes[i],init_fn=init_fn,act_fn=act_fn))
        self.layers.append(Layer(dim_in=hidden_sizes[-1],dim_out=output_size,init_fn=init_fn,act_fn=False))

    

    def forward(self,input):
        output=input
        for layer in self.layers:
            output=layer.fw(output)
        return output

    def print_info(self):
        for layer in self.layers:
            print("weights")
            print(layer.weights)
            print("bias")
            print(layer.bias)





In [6]:
test=FFNN(input_size=3,hidden_sizes=[2,2,4,4],output_size=3,init_fn=zeros,act_fn=Relu)

In [7]:
test_input=np.array([3,14,5])


In [8]:
test.print_info()

weights
[[0. 0. 0.]
 [0. 0. 0.]]
bias
[0. 0.]
weights
[[0. 0.]
 [0. 0.]]
bias
[0. 0.]
weights
[[0. 0.]
 [0. 0.]
 [0. 0.]
 [0. 0.]]
bias
[0. 0. 0. 0.]
weights
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
bias
[0. 0. 0. 0.]
weights
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
bias
[0. 0. 0.]


In [9]:
print(test.forward(test_input))

[0. 0. 0.]
